In [1]:
### DO NOT RUN THIS.  It builds the BkAvgAst... tables, which are supplied in the processing tables zip file.
### 
### NOTE:  Major change due to conversion of some bank holding companies back to bank structures.
###        (This removes the Parent (NAMEHCR) and ParRSSD (RSSDHCR) fields from the institutions2.zip listing.)
###        

import numpy as np
import pandas as pd
from datetime import datetime
#import pandas_datareader.data as web
import warnings
warnings.filterwarnings('ignore')

import matplotlib as mpl
import matplotlib.pyplot as plt
%matplotlib inline

import os
os.chdir('C:/BankRisk')


In [2]:
rk = pd.read_csv("BkAvgAstPR_FailDate_202003.csv", index_col=0)
rk['xAsOf'] = pd.to_datetime(rk['xAsOf'])
rk['nAsOf'] = pd.to_datetime(rk['nAsOf'])
rk['FailDate'] = pd.to_datetime(rk['FailDate'])
rk.head()

Rank  IDRSSD      xAsOf                    xName xState    xCert  \
0     1  852218 2020-03-31  JPMORGAN CHASE BANK, NA     OH    628.0   
1     2  480228 2020-03-31      BANK OF AMERICA, NA     NC   3510.0   
2     3  476810 2020-03-31           CITIBANK, N.A.     SD   7213.0   
3     4  451965 2020-03-31     WELLS FARGO BANK, NA     SD   3511.0   
4     5  484422 2009-12-31        WACHOVIA BANK, NA     NC  33869.0   

       nAsOf      aeTotAst      aaTotAst       aeTotLn        aeDDep  \
0 2001-03-31  1.542173e+09  1.516049e+09  5.482606e+08  7.277456e+08   
1 2001-03-31  1.307893e+09  1.308298e+09  6.849586e+08  8.495753e+08   
2 2001-03-31  1.090942e+09  1.086881e+09  5.184697e+08  3.028914e+08   
3 2001-03-31  9.457972e+08  9.340876e+08  5.642137e+08  6.660571e+08   
4 2001-03-31  4.416464e+08  4.372459e+08  2.546369e+08  2.704156e+08   

         aeFDep      aeTotDep   FailDate    pr_eTA   pr_eTAcs    pr_eTL  \
0  2.576001e+08  9.853457e+08        NaT  9.254091   9.254091  5.770519   
1  1.043175e+08  9.538928e+08 2009-01-16  7.848250  17.102340  7.209284   
2  4.486076e+08  7.514990e+08 2008-11-23  6.546401  23.648741  5.456966   
3  6.632369e+07  7.323808e+08        NaT  5.675430  29.324171  5.938428   
4  2.469325e+07  2.951088e+08        NaT  2.650180  31.974351  2.680089   

    pr_eTLcs    pr_eDD   pr_eDDcs  
0   5.770519  7.659623   7.659623  
1  12.979803  8.941898  16.601521  
2  18.436768  3.187974  19.789496  
3  24.375196  7.010344  26.799839  
4  27.055285  2.846162  29.646001

In [3]:
len(rk) # 10792

10792

In [4]:
bhc = pd.read_csv("INSTITUTIONS2_202004.csv", encoding='latin-1') # Warning: This is latest available file as of 2019-April.
bhc.head()

STNAME  STMULT  CERT  DOCKET  ACTIVE              ADDRESS      ASSET  \
0  Connecticut     NaN     4       0       0       22 Main Street        NaN   
1  Connecticut     NaN     6       0       0  81 West Main Street        NaN   
2        Maine     NaN     8       0       0      One City Center  1,699,404   
3        Maine     NaN     9    9781       0       66 Main Street    539,169   
4        Maine     NaN    10       0       0  8 Washington Street        NaN   

  BKCLASS  CHANGEC1  CHANGEC2  ...                                   CBSA  \
0      NM       223         0  ...  Hartford-East Hartford-Middletown, CT   
1      NM       223       999  ...                  New Haven-Milford, CT   
2      SM       223         0  ...            Portland-South Portland, ME   
3      SM       223         0  ...                                    NaN   
4      SM       223         0  ...            Portland-South Portland, ME   

   CBSA_NO                        CBSA_METRO_NAME  CBSA_METRO CBSA_METRO_FLG  \
0    25540  Hartford-East Hartford-Middletown, CT       25540              1   
1    35300                  New Haven-Milford, CT       35300              1   
2    38860            Portland-South Portland, ME       38860              1   
3        0                                    NaN           0              0   
4    38860            Portland-South Portland, ME       38860              1   

  CBSA_MICRO_FLG CBSA_DIV  CBSA_DIV_NO  CBSA_DIV_FLG  CB  
0              0      NaN            0             0 NaN  
1              0      NaN            0             0 NaN  
2              0      NaN            0             0 NaN  
3              0      NaN            0             0 NaN  
4              0      NaN            0             0 NaN  

[5 rows x 134 columns]

In [5]:
bhc.replace(to_replace='orporated', value='orp', inplace=True, regex=True)
bhc.replace(to_replace='Irrevocable', value='Irrev', inplace=True, regex=True)
bhc.replace(to_replace='Employee Stock Ownership ', value='ESO', inplace=True, regex=True)
bhc.replace(to_replace='orporation', value='orp', inplace=True, regex=True)
bhc.replace(to_replace='Financial', value='Fin', inplace=True, regex=True)
bhc.replace(to_replace='Services', value='Serv', inplace=True, regex=True)
bhc.replace(to_replace='Holding Company', value='HC', inplace=True, regex=True)
#bhc.replace(to_replace=', Inc.', value=' Inc', inplace=True, regex=True)
bhc.replace(to_replace='Inc.', value='Inc', inplace=True, regex=True)
bhc.replace(to_replace='Corp.', value='Corp', inplace=True, regex=True)
bhc.replace(to_replace=', The', value=' The', inplace=True, regex=True)
bhc.replace(to_replace='orpThe', value='orp', inplace=True, regex=True)

In [6]:
len(bhc) # 27623

27623

In [7]:
bhc.head(2)

STNAME  STMULT  CERT  DOCKET  ACTIVE              ADDRESS ASSET  \
0  Connecticut     NaN     4       0       0       22 Main Street   NaN   
1  Connecticut     NaN     6       0       0  81 West Main Street   NaN   

  BKCLASS  CHANGEC1  CHANGEC2  ...                                   CBSA  \
0      NM       223         0  ...  Hartford-East Hartford-Middletown, CT   
1      NM       223       999  ...                  New Haven-Milford, CT   

   CBSA_NO                        CBSA_METRO_NAME  CBSA_METRO CBSA_METRO_FLG  \
0    25540  Hartford-East Hartford-Middletown, CT       25540              1   
1    35300                  New Haven-Milford, CT       35300              1   

  CBSA_MICRO_FLG CBSA_DIV  CBSA_DIV_NO  CBSA_DIV_FLG  CB  
0              0      NaN            0             0 NaN  
1              0      NaN            0             0 NaN  

[2 rows x 134 columns]

In [8]:
bhc2=bhc[['CERT','FED_RSSD','NAME','STALP','BKCLASS','OFFICES','NAMEHCR','RSSDHCR','STALPHCR','ULTCERT']]
#bhc2.columns = [['Cert','IDRSSD','Name','ST','BkClass','Offices','Parent','ParRSSD','ParST','UltCert']]
bhc2.rename(index=str, columns={"CERT":"Cert", "FED_RSSD":"IDRSSD", "NAME":"Name", "STALP":"ST", "BKCLASS":"BkClass", \
            "OFFICES":"Offices", "NAMEHCR":"Parent", "RSSDHCR":"ParRSSD", "STALPHCR":"ParST", "ULTCERT":"UltCert"}, inplace=True)
len(bhc2) # 27623

27623

In [9]:
bhc2.head(2)

Cert  IDRSSD                                    Name  ST BkClass  Offices  \
0     4  573401  The Southington Bank and Trust Company  CT      NM      4.0   
1     6  148304              Colonial Bank of Waterbury  CT      NM     34.0   

  Parent  ParRSSD ParST  UltCert  
0    NaN      NaN   NaN     3510  
1    NaN      NaN   NaN     3510

In [10]:
bhc2.columns

Index(['Cert', 'IDRSSD', 'Name', 'ST', 'BkClass', 'Offices', 'Parent',
       'ParRSSD', 'ParST', 'UltCert'],
      dtype='object')

In [11]:
bhc2[bhc2['Parent']>' '][0:10]

Cert  IDRSSD                                 Name  ST BkClass  Offices  \
5     14   35301  State Street Bank and Trust Company  MA      SM      3.0   
15    35  749635                           AuburnBank  AL      SM      8.0   
19    39  430036            Robertson Banking Company  AL      NM      7.0   
21    41  432432                   Phenix-Girard Bank  AL      NM      4.0   
25    46  113937            Merchants Bank of Alabama  AL      NM      5.0   
26    47  390336               Traders & Farmers Bank  AL      NM      7.0   
29    50  470032                Community Spirit Bank  AL      NM      5.0   
30    51  382537                   The Bank of Vernon  AL      NM      4.0   
31    52  293437        The Citizens Bank of Winfield  AL      NM      1.0   
33    54  672537         The Exchange Bank of Alabama  AL      NM      5.0   

                            Parent    ParRSSD ParST  UltCert  
5                State Street Corp  1111435.0    MA       14  
15       Auburn National Bcorp Inc  1129533.0    AL       35  
19                         Rbc Inc  2526755.0    AL       39  
21    Phenix-Girard Bancshares Inc  2013246.0    AL       41  
25         Merchants Finl Serv Inc  1082955.0    AL       46  
26  Traders&Farmers Bancshares Inc  3467305.0    AL       47  
29  Independent Bancshares Employe  3587294.0    AL       50  
30     First Vernon Bancshares Inc  1139998.0    AL       51  
31  Citizens Bcorp Of Winfield Inc  1833616.0    AL       52  
33                    Gadsden Corp  1079067.0    AL       54

In [12]:
#bhc2[(bhc2['ParRSSD']>0)&pd.isnull(bhc2['Parent'])]  ## should be null list

In [13]:
#bhc2[(bhc2['UltCert']>0)&pd.isnull(bhc2['Parent'])].head()

In [14]:
## extract ticker info from prior listing: (rkhn was expanded output from br001f_NewBHCtickers code)
rkh = pd.read_csv("BkAvgAstPR_FailDate_BHC2_ult201903.csv", index_col=0)

#len(rkh) # 10775
len(rkh) # 10776

10776

In [15]:
len(rkh[rkh['Ticker']>'']) # 3266

3266

In [16]:
#uex = hs3[['Date','UE','UE_'+st]].ix[hs3['UE'].idxmax()]
rkh2 = rkh[rkh['ParRSSD']>0][['Rank','ParRSSD','ParST','Ticker']]
len(rkh2) # 8794

8794

In [17]:
rkh2.tail(2)

Rank    ParRSSD ParST Ticker
10772  10773  1094314.0    MO  CBCYB
10775  10776  3447567.0    TX    NaN

In [18]:
rkh3 = rkh2.groupby(['ParRSSD','Ticker']).min()
rkh3.sort_index(by='Rank',inplace=True)
len(rkh3) # 402

402

In [19]:
rkh3.head()

Rank ParST
ParRSSD   Ticker            
1039502.0 JPM        1    NY
1073757.0 BAC        2    NC
1951350.0 C          3    NY
1120754.0 WFC        4    CA
1119794.0 USB        6    MN

In [20]:
rkh4 = rkh3.reset_index(drop=False)
rkh4.head()

ParRSSD Ticker  Rank ParST
0  1039502.0    JPM     1    NY
1  1073757.0    BAC     2    NC
2  1951350.0      C     3    NY
3  1120754.0    WFC     4    CA
4  1119794.0    USB     6    MN

In [21]:
rkh5=rkh4[['ParRSSD','Ticker']].groupby('Ticker').count()
rkh5.head()

ParRSSD
Ticker         
ABCB          1
ABTX          1
ABYB          1
ACBI          1
ACNB          1

In [22]:
rkh5[rkh5['ParRSSD']>1] # finds tickers with more than one ParRSSD code:

Empty DataFrame
Columns: [ParRSSD]
Index: []

In [23]:
rkh[rkh['Ticker']=="BHWB"].iloc[:,0:12] # check on Ticker that was misidentified in earlier runs (should be 2 banks only)

Rank  IDRSSD       xAsOf           xName xState    xCert       nAsOf  \
1737  1738   27548  2019-03-31  BLACKHAWK BANK     WI  14078.0  2001-03-31   
7747  7748  388931  2003-09-30   FIRST BANK BC     IL  16551.0  2001-03-31   

     FailDate BkClass  Offices                 Parent    ParRSSD  
1737      NaN      NM      9.0  Blackhawk Bancorp Inc  1491913.0  
7747      NaN      NM      4.0  Blackhawk Bancorp Inc  1491913.0

In [24]:
bhc3 = bhc2.merge(rkh4[['ParRSSD','Ticker']],how='left',on='ParRSSD')
bhc3.head()

Cert  IDRSSD                                    Name  ST BkClass  Offices  \
0     4  573401  The Southington Bank and Trust Company  CT      NM      4.0   
1     6  148304              Colonial Bank of Waterbury  CT      NM     34.0   
2     8  422406                     Fleet Bank of Maine  ME      SM     55.0   
3     9  563907                     Union Trust Company  ME      SM     13.0   
4    10  112109               Northeast Bank of Sanford  ME      SM      5.0   

  Parent  ParRSSD ParST  UltCert Ticker  
0    NaN      NaN   NaN     3510    NaN  
1    NaN      NaN   NaN     3510    NaN  
2    NaN      NaN   NaN     3510    NaN  
3    NaN      NaN   NaN     4255    NaN  
4    NaN      NaN   NaN     3510    NaN

In [25]:
bhc3[bhc3['Cert']==3510]

Cert  IDRSSD                                   Name  ST BkClass  \
2233  3510  480228  Bank of America, National Association  NC       N   

      Offices                Parent    ParRSSD ParST  UltCert Ticker  
2233   4263.0  Bank Of America Corp  1073757.0    NC     3510    BAC

In [26]:
bhc2[bhc2['Cert']==3510]

Cert  IDRSSD                                   Name  ST BkClass  \
2233  3510  480228  Bank of America, National Association  NC       N   

      Offices                Parent    ParRSSD ParST  UltCert  
2233   4263.0  Bank Of America Corp  1073757.0    NC     3510

In [27]:
rk2 = rk.merge(bhc2[['Cert','BkClass','Offices','Parent','ParRSSD','ParST','UltCert']],how='left',left_on='xCert',\
              right_on='Cert')
rk2.head(2)

Rank  IDRSSD      xAsOf                    xName xState   xCert      nAsOf  \
0     1  852218 2020-03-31  JPMORGAN CHASE BANK, NA     OH   628.0 2001-03-31   
1     2  480228 2020-03-31      BANK OF AMERICA, NA     NC  3510.0 2001-03-31   

       aeTotAst      aaTotAst       aeTotLn  ...   pr_eTLcs    pr_eDD  \
0  1.542173e+09  1.516049e+09  5.482606e+08  ...   5.770519  7.659623   
1  1.307893e+09  1.308298e+09  6.849586e+08  ...  12.979803  8.941898   

    pr_eDDcs    Cert  BkClass  Offices                Parent    ParRSSD  \
0   7.659623   628.0        N   5055.0     Jpmorgan Chase&Co  1039502.0   
1  16.601521  3510.0        N   4263.0  Bank Of America Corp  1073757.0   

   ParST  UltCert  
0     NY    628.0  
1     NC   3510.0  

[2 rows x 27 columns]

In [28]:
rkh4.head(2)

ParRSSD Ticker  Rank ParST
0  1039502.0    JPM     1    NY
1  1073757.0    BAC     2    NC

In [29]:
rk3 = rk2.merge(rkh4[['ParRSSD','Ticker']],how='left',on='ParRSSD')
rk3.head(2)

Rank  IDRSSD      xAsOf                    xName xState   xCert      nAsOf  \
0     1  852218 2020-03-31  JPMORGAN CHASE BANK, NA     OH   628.0 2001-03-31   
1     2  480228 2020-03-31      BANK OF AMERICA, NA     NC  3510.0 2001-03-31   

       aeTotAst      aaTotAst       aeTotLn  ...    pr_eDD   pr_eDDcs    Cert  \
0  1.542173e+09  1.516049e+09  5.482606e+08  ...  7.659623   7.659623   628.0   
1  1.307893e+09  1.308298e+09  6.849586e+08  ...  8.941898  16.601521  3510.0   

  BkClass  Offices                Parent    ParRSSD  ParST  UltCert  Ticker  
0       N   5055.0     Jpmorgan Chase&Co  1039502.0     NY    628.0     JPM  
1       N   4263.0  Bank Of America Corp  1073757.0     NC   3510.0     BAC  

[2 rows x 28 columns]

In [30]:
rk3c = list(rk3.columns)
rk3c2 = rk3c[0:7] + ['FailDate'] + rk3c[-7:] + rk3c[7:13] + rk3c[14:20]
rk3c, rk3c2

(['Rank',
  'IDRSSD',
  'xAsOf',
  'xName',
  'xState',
  'xCert',
  'nAsOf',
  'aeTotAst',
  'aaTotAst',
  'aeTotLn',
  'aeDDep',
  'aeFDep',
  'aeTotDep',
  'FailDate',
  'pr_eTA',
  'pr_eTAcs',
  'pr_eTL',
  'pr_eTLcs',
  'pr_eDD',
  'pr_eDDcs',
  'Cert',
  'BkClass',
  'Offices',
  'Parent',
  'ParRSSD',
  'ParST',
  'UltCert',
  'Ticker'],
 ['Rank',
  'IDRSSD',
  'xAsOf',
  'xName',
  'xState',
  'xCert',
  'nAsOf',
  'FailDate',
  'BkClass',
  'Offices',
  'Parent',
  'ParRSSD',
  'ParST',
  'UltCert',
  'Ticker',
  'aeTotAst',
  'aaTotAst',
  'aeTotLn',
  'aeDDep',
  'aeFDep',
  'aeTotDep',
  'pr_eTA',
  'pr_eTAcs',
  'pr_eTL',
  'pr_eTLcs',
  'pr_eDD',
  'pr_eDDcs'])

In [31]:
rk4 = rk3[rk3c2]
rk4.head(2)

Rank  IDRSSD      xAsOf                    xName xState   xCert      nAsOf  \
0     1  852218 2020-03-31  JPMORGAN CHASE BANK, NA     OH   628.0 2001-03-31   
1     2  480228 2020-03-31      BANK OF AMERICA, NA     NC  3510.0 2001-03-31   

    FailDate BkClass  Offices  ...       aeTotLn        aeDDep        aeFDep  \
0        NaT       N   5055.0  ...  5.482606e+08  7.277456e+08  2.576001e+08   
1 2009-01-16       N   4263.0  ...  6.849586e+08  8.495753e+08  1.043175e+08   

       aeTotDep    pr_eTA   pr_eTAcs    pr_eTL   pr_eTLcs    pr_eDD   pr_eDDcs  
0  9.853457e+08  9.254091   9.254091  5.770519   5.770519  7.659623   7.659623  
1  9.538928e+08  7.848250  17.102340  7.209284  12.979803  8.941898  16.601521  

[2 rows x 27 columns]

In [32]:
## New code (May 2019) to add holding company values for Parent, ParRSSD, Ticker for Zions Bancorp (since this data)
## was recently removed due to legal changes to terminate Fed CCAR oversight
## If other companies do the same, their data will be added back here
## (This is essential for the historical CCAR 31-bank group to compute properly)

In [33]:
rk4.loc[rk4.xCert.isin([2270]),['IDRSSD','xName','xCert','Ticker','Parent','ParRSSD']]

IDRSSD                     xName   xCert Ticker Parent  ParRSSD
78  276579  ZIONS BANCORPORATION, NA  2270.0    NaN    NaN      NaN

In [34]:
rk4.loc[rk4.xCert == 2270, 'Parent'] = 'ZION'
rk4.loc[rk4.xCert == 2270, 'Ticker'] = 'ZION'
rk4.loc[rk4.xCert == 2270, 'ParRSSD'] = 1027004.0

In [35]:
rk4.loc[rk4.xCert.isin([2270]),['IDRSSD','xName','xCert','Ticker','Parent','ParRSSD']]

IDRSSD                     xName   xCert Ticker Parent    ParRSSD
78  276579  ZIONS BANCORPORATION, NA  2270.0   ZION   ZION  1027004.0

In [36]:
rk4.to_csv("BkAvgAstPR_FailDate_BHC_nohist202003.csv")

In [37]:
len(rk4) # 10792

10792

In [38]:
## find bhcs with totast above 700MM and no ticker:
rkm = rk4[(rk4['ParRSSD']>0)&(rk4['aeTotAst']>700000)&pd.isnull(rk4['Ticker'])]
len(rkm) # 270

270

In [39]:
rkm.head()

Rank   IDRSSD      xAsOf                      xName xState    xCert  \
21    22  3150447 2020-03-31   CHARLES SCHWAB BANK, SSB     TX  57450.0   
33    34   619877 2020-03-31  USAA FEDERAL SAVINGS BANK     TX  32188.0   
50    51  1216022 2020-03-31             SYNCHRONY BANK     UT  27314.0   
54    55   370271 2020-03-31               E*TRADE BANK     VA  30746.0   
77    78  2735146 2020-03-31                  TIAA, FSB     FL  34775.0   

        nAsOf FailDate BkClass  Offices  ...       aeTotLn        aeDDep  \
21 2003-06-30      NaT      SM      1.0  ...  9.901230e+06  9.189240e+07   
33 2012-03-31      NaT      SA      1.0  ...  4.438287e+07  6.524968e+07   
50 2001-03-31      NaT      SA      4.0  ...  4.159228e+07  3.620739e+07   
54 2012-03-31      NaT      SA      1.0  ...  8.408220e+06  3.647944e+07   
77 2012-03-31      NaT      SA     14.0  ...  2.294125e+07  1.881568e+07   

      aeFDep      aeTotDep    pr_eTA   pr_eTAcs    pr_eTL   pr_eTLcs  \
21   0.00000  9.189240e+07  0.600433  49.020072  0.104212  41.274332   
33   0.00000  6.524968e+07  0.440975  55.155592  0.467136  48.239891   
50  30.04878  3.620742e+07  0.296461  61.312666  0.437764  55.620454   
54   0.00000  3.647944e+07  0.266102  62.389797  0.088498  56.730164   
77   0.00000  1.881568e+07  0.162902  67.066840  0.241460  62.318401   

      pr_eDD   pr_eDDcs  
21  0.967180  46.872175  
33  0.686762  53.800162  
50  0.381088  60.660832  
54  0.383951  61.966989  
77  0.198038  66.972268  

[5 rows x 27 columns]

In [40]:
rkm.to_csv('rk_missingtickers202003.csv')

In [41]:
##############################################################################

In [42]:
## start up here after filling in missing tickers

In [43]:
## filled in many missing tickers

In [44]:
rkm2 = pd.read_csv('rk_missingtickers202003a.csv',index_col=0)
rkm2.head()

Rank   IDRSSD      xAsOf                      xName xState  xCert  \
21    22  3150447  3/31/2020   CHARLES SCHWAB BANK, SSB     TX  57450   
33    34   619877  3/31/2020  USAA FEDERAL SAVINGS BANK     TX  32188   
50    51  1216022  3/31/2020             SYNCHRONY BANK     UT  27314   
54    55   370271  3/31/2020               E*TRADE BANK     VA  30746   
77    78  2735146  3/31/2020                  TIAA, FSB     FL  34775   

        nAsOf FailDate BkClass  Offices  ...       aeTotLn       aeDDep  \
21  6/30/2003      NaN      SM        1  ...  9.901230e+06  91892404.33   
33  3/31/2012      NaN      SA        1  ...  4.438287e+07  65249683.52   
50  3/31/2001      NaN      SA        4  ...  4.159228e+07  36207388.66   
54  3/31/2012      NaN      SA        1  ...  8.408220e+06  36479437.52   
77  3/31/2012      NaN      SA       14  ...  2.294125e+07  18815675.30   

      aeFDep     aeTotDep    pr_eTA   pr_eTAcs    pr_eTL   pr_eTLcs    pr_eDD  \
21   0.00000  91892404.33  0.600433  49.020072  0.104212  41.274332  0.967180   
33   0.00000  65249683.52  0.440975  55.155592  0.467136  48.239891  0.686762   
50  30.04878  36207418.71  0.296461  61.312666  0.437764  55.620454  0.381088   
54   0.00000  36479437.52  0.266102  62.389797  0.088498  56.730164  0.383951   
77   0.00000  18815675.30  0.162902  67.066840  0.241460  62.318401  0.198038   

     pr_eDDcs  
21  46.872175  
33  53.800162  
50  60.660832  
54  61.966989  
77  66.972268  

[5 rows x 27 columns]

In [45]:
rkm3 = rkm2[rkm2['Ticker']>' '][['ParRSSD','Ticker']]
rkm3.head()

ParRSSD Ticker
21   1026632   SCHW
50   4504654    SYF
54   3412583   ETFC
79   1232497     RY
117  4224000    FBC

In [46]:
rkm3p = set(rkm3['ParRSSD'])
len(rkm3), len(rkm3p) # 34, 30

(34, 30)

In [47]:
rk4[rk4['ParRSSD'].isin(rkm3p)][['xName','Parent','Ticker']]

xName                         Parent  \
21                CHARLES SCHWAB BANK, SSB            Charles Schwab Corp   
50                          SYNCHRONY BANK                  Synchrony Fin   
54                            E*TRADE BANK               E*trade Fin Corp   
79                      CITY NATIONAL BANK           Royal Bank Of Canada   
117                     FLAGSTAR BANK, FSB                  Mp Thrift Llc   
121                    FIRST HAWAIIAN BANK             First Hawaiian Inc   
172           CAPITOL FEDERAL SAVINGS BANK        Capitol Federal Fin Inc   
205                         NORTHWEST BANK       Northwest Bancshares Inc   
210                             FIRST BANK                        Fb Corp   
219                              AXOS BANK                   Axos Fin Inc   
225   WILMINGTON SAVINGS FUND SOCIETY, FSB                  Wsfs Fin Corp   
239          AMERICAN SAVINGS BANK, F.S.B.   Hawaiian Electric Industries   
245                       AMERANT BANK, NA              Amerant Bcorp Inc   
257                     PLAINSCAPITAL BANK    Diamond Hth Stock Co Gp Llc   
267                    DIME COMMUNITY BANK  Dime Community Bancshares Inc   
290                           TRUSTCO BANK            Trustco Bank CorpNy   
301                            KEARNY BANK                Kearny Fin Corp   
325                      MIZUHO BANK (USA)           Mizuho Fin Group Inc   
330       CHARLES SCHWAB PREMIER BANK, SSB            Charles Schwab Corp   
346              CHARLES SCHWAB TRUST BANK            Charles Schwab Corp   
352                               METABANK             Meta Fin Group Inc   
432           JOHN DEERE FINANCIAL, F.S.B.              Deere&Company Inc   
435                         HARBORONE BANK            Harborone Bcorp Inc   
436      FIRST FEDERAL BANK OF THE MIDWEST        First Defiance Fin Corp   
444                 RBC BANK (GEORGIA), NA           Royal Bank Of Canada   
447                    ETRADE SAVINGS BANK               E*trade Fin Corp   
454                        NORTHFIELD BANK           Northfield Bcorp Inc   
494                            ALPINE BANK       Alpine Banks Of Colorado   
569           STERLING BANK AND TRUST, FSB             Sterling Bcorp Inc   
593               TERRITORIAL SAVINGS BANK          Territorial Bcorp Inc   
621       NORTH AMERICAN SAVINGS BANK, FSB                   Nasb Fin Inc   
640                        WATERSTONE BANK             Waterstone Fin Inc   
690                        CROSSFIRST BANK      Crossfirst Bankshares Inc   
721                         WESTFIELD BANK  Western New England Bcorp Inc   
2706                   FIRST GULF BANK, NA           Royal Bank Of Canada   

     Ticker  
21      NaN  
50      NaN  
54      NaN  
79      NaN  
117     NaN  
121     NaN  
172     NaN  
205     NaN  
210     NaN  
219     NaN  
225     NaN  
239     NaN  
245     NaN  
257     NaN  
267     NaN  
290     NaN  
301     NaN  
325     NaN  
330     NaN  
346     NaN  
352     NaN  
432     NaN  
435     NaN  
436     NaN  
444     NaN  
447     NaN  
454     NaN  
494     NaN  
569     NaN  
593     NaN  
621     NaN  
640     NaN  
690     NaN  
721     NaN  
2706    NaN

In [48]:
rk4c = rk4.copy()

In [49]:
tdct = dict(zip(rkm3['ParRSSD'],rkm3['Ticker']))
tdct

{1026632: 'SCHW',
 4504654: 'SYF',
 3412583: 'ETFC',
 1232497: 'RY',
 4224000: 'FBC',
 1025608: 'FHB',
 4226910: 'CFFN',
 4122722: 'NWBI',
 1118797: 'FRBA',
 3814310: 'AX',
 3844269: 'WSFS',
 3842957: 'HE',
 1135972: 'AMTB',
 5213690: 'HTH',
 2487650: 'DCOM',
 1048513: 'TRST',
 3099443: 'KRNY',
 3185355: 'MFG',
 2390013: 'CASH',
 1277313: 'DE',
 4973353: 'HONE',
 3316917: 'FDEF',
 3132863: 'NFBK',
 1061679: 'ALPIB',
 3852031: 'SBT',
 3954681: 'TBNK',
 3841941: 'NASB',
 4523431: 'WSBF',
 3903661: 'CFB',
 3866382: 'WNEB'}

In [50]:
## update missing Tickers if it exists:
for i in np.arange(len(rk4)):
  rs = rk4.ix[i]['ParRSSD']
  try:
    rk4.loc[i,'Ticker'] = tdct[rs]
  except:
    pass

In [51]:
rk4[rk4['ParRSSD'].isin(rkm3p)][['xName','Parent','ParRSSD','Ticker']]

xName                         Parent  \
21                CHARLES SCHWAB BANK, SSB            Charles Schwab Corp   
50                          SYNCHRONY BANK                  Synchrony Fin   
54                            E*TRADE BANK               E*trade Fin Corp   
79                      CITY NATIONAL BANK           Royal Bank Of Canada   
117                     FLAGSTAR BANK, FSB                  Mp Thrift Llc   
121                    FIRST HAWAIIAN BANK             First Hawaiian Inc   
172           CAPITOL FEDERAL SAVINGS BANK        Capitol Federal Fin Inc   
205                         NORTHWEST BANK       Northwest Bancshares Inc   
210                             FIRST BANK                        Fb Corp   
219                              AXOS BANK                   Axos Fin Inc   
225   WILMINGTON SAVINGS FUND SOCIETY, FSB                  Wsfs Fin Corp   
239          AMERICAN SAVINGS BANK, F.S.B.   Hawaiian Electric Industries   
245                       AMERANT BANK, NA              Amerant Bcorp Inc   
257                     PLAINSCAPITAL BANK    Diamond Hth Stock Co Gp Llc   
267                    DIME COMMUNITY BANK  Dime Community Bancshares Inc   
290                           TRUSTCO BANK            Trustco Bank CorpNy   
301                            KEARNY BANK                Kearny Fin Corp   
325                      MIZUHO BANK (USA)           Mizuho Fin Group Inc   
330       CHARLES SCHWAB PREMIER BANK, SSB            Charles Schwab Corp   
346              CHARLES SCHWAB TRUST BANK            Charles Schwab Corp   
352                               METABANK             Meta Fin Group Inc   
432           JOHN DEERE FINANCIAL, F.S.B.              Deere&Company Inc   
435                         HARBORONE BANK            Harborone Bcorp Inc   
436      FIRST FEDERAL BANK OF THE MIDWEST        First Defiance Fin Corp   
444                 RBC BANK (GEORGIA), NA           Royal Bank Of Canada   
447                    ETRADE SAVINGS BANK               E*trade Fin Corp   
454                        NORTHFIELD BANK           Northfield Bcorp Inc   
494                            ALPINE BANK       Alpine Banks Of Colorado   
569           STERLING BANK AND TRUST, FSB             Sterling Bcorp Inc   
593               TERRITORIAL SAVINGS BANK          Territorial Bcorp Inc   
621       NORTH AMERICAN SAVINGS BANK, FSB                   Nasb Fin Inc   
640                        WATERSTONE BANK             Waterstone Fin Inc   
690                        CROSSFIRST BANK      Crossfirst Bankshares Inc   
721                         WESTFIELD BANK  Western New England Bcorp Inc   
2706                   FIRST GULF BANK, NA           Royal Bank Of Canada   

        ParRSSD Ticker  
21    1026632.0   SCHW  
50    4504654.0    SYF  
54    3412583.0   ETFC  
79    1232497.0     RY  
117   4224000.0    FBC  
121   1025608.0    FHB  
172   4226910.0   CFFN  
205   4122722.0   NWBI  
210   1118797.0   FRBA  
219   3814310.0     AX  
225   3844269.0   WSFS  
239   3842957.0     HE  
245   1135972.0   AMTB  
257   5213690.0    HTH  
267   2487650.0   DCOM  
290   1048513.0   TRST  
301   3099443.0   KRNY  
325   3185355.0    MFG  
330   1026632.0   SCHW  
346   1026632.0   SCHW  
352   2390013.0   CASH  
432   1277313.0     DE  
435   4973353.0   HONE  
436   3316917.0   FDEF  
444   1232497.0     RY  
447   3412583.0   ETFC  
454   3132863.0   NFBK  
494   1061679.0  ALPIB  
569   3852031.0    SBT  
593   3954681.0   TBNK  
621   3841941.0   NASB  
640   4523431.0   WSBF  
690   3903661.0    CFB  
721   3866382.0   WNEB  
2706  1232497.0     RY

In [52]:
len(rk4[rk4['Ticker']>' ']),len(rk4c[rk4c['Ticker']>' ']) # 512, 477

(512, 477)

In [53]:
rk4.to_csv("BkAvgAstPR_FailDate_BHC2_nohist202003.csv")

In [54]:
## process ultcert version:

In [55]:
rk5 = rk4.copy()

In [56]:
ult = set(rk5['UltCert'])
len(ult) # 5472

5472

In [57]:
bhcc = rk5[rk5['xCert'].isin(ult)][['xCert','Parent','Ticker','ParRSSD','ParST']]
bhcc.head()

xCert                Parent Ticker    ParRSSD ParST
0   628.0     Jpmorgan Chase&Co    JPM  1039502.0    NY
1  3510.0  Bank Of America Corp    BAC  1073757.0    NC
2  7213.0         Citigroup Inc      C  1951350.0    NY
3  3511.0   Wells Fargo&Company    WFC  1120754.0    CA
5  6548.0             U S Bcorp    USB  1119794.0    MN

In [58]:
bhcc.dropna(subset=['Parent'],inplace=True)
len(bhcc) # 4190

4190

In [59]:
len(bhcc[bhcc['Ticker']>'']) # 505

505

In [60]:
bhcc.head()

xCert                Parent Ticker    ParRSSD ParST
0   628.0     Jpmorgan Chase&Co    JPM  1039502.0    NY
1  3510.0  Bank Of America Corp    BAC  1073757.0    NC
2  7213.0         Citigroup Inc      C  1951350.0    NY
3  3511.0   Wells Fargo&Company    WFC  1120754.0    CA
5  6548.0             U S Bcorp    USB  1119794.0    MN

In [61]:
bhcc.groupby('xCert').filter(lambda x: len(x) > 1)  ## check for duplicates in xCert index

xCert                          Parent Ticker    ParRSSD ParST
37     16571.0                Bank Of Montreal    BMO  1231333.0     0
176     4999.0                   Cadence Bcorp   CADE  4882802.0    TX
252    58564.0               Flushing Fin Corp   FFIC  2393274.0    NY
444    26342.0            Royal Bank Of Canada     RY  1232497.0     0
678    35525.0     Atlantic Capital Bancshares   ACBI  3555686.0    GA
688     4999.0                   Cadence Bcorp   CADE  4882802.0    TX
1029   35525.0     Atlantic Capital Bancshares   ACBI  3555686.0    GA
1067    8888.0                    Ccb Fin Corp    NaN  1053496.0    MO
1203   57197.0                     Bnccorp Inc   BNCC  1248162.0    ND
1549    3116.0  Lone Star State Bancshares Inc    NaN  3641767.0    TX
1746   10973.0          Alliance Finl Serv Inc    NaN  1127146.0    MN
1804      89.0       White River Bancshares Co    NaN  3306589.0    AR
1834   16571.0                Bank Of Montreal    BMO  1231333.0     0
1918   57497.0            Pathfinder Bcorp Inc    NaN  2596776.0    NY
2233   58564.0               Flushing Fin Corp   FFIC  2393274.0    NY
2681   13198.0      Cornerstone Holding Co Inc    NaN  3435283.0    ND
2706   26342.0            Royal Bank Of Canada     RY  1232497.0     0
3935   35274.0       Blue Ridge Bankshares Inc    NaN  1426960.0    VA
4078   11310.0                   Fgh Bcorp Inc    NaN  2527024.0    IL
4413   13198.0      Cornerstone Holding Co Inc    NaN  3435283.0    ND
4555    8252.0        Jamesmark Bancshares Inc    NaN  2816803.0    MO
4559   17636.0     First Belleville Bancshares    NaN  1060917.0    KS
6019    3116.0  Lone Star State Bancshares Inc    NaN  3641767.0    TX
6080    1756.0     Cross County Bancshares Inc    NaN  1131611.0    AR
6777   17399.0                  First Fin Corp    NaN  1067514.0    KS
7160      89.0       White River Bancshares Co    NaN  3306589.0    AR
7163   10973.0          Alliance Finl Serv Inc    NaN  1127146.0    MN
7425   13381.0  Schuyler County Bancshares Inc    NaN  1100055.0    MO
7549   35274.0       Blue Ridge Bankshares Inc    NaN  1426960.0    VA
7813    8252.0        Jamesmark Bancshares Inc    NaN  2816803.0    MO
8665   29617.0         First Mutual Holding Co    NaN  4843090.0    OH
8809   57497.0            Pathfinder Bcorp Inc    NaN  2596776.0    NY
9105   29617.0         First Mutual Holding Co    NaN  4843090.0    OH
9151   11310.0                   Fgh Bcorp Inc    NaN  2527024.0    IL
9693    8888.0                    Ccb Fin Corp    NaN  1053496.0    MO
9819   17399.0                  First Fin Corp    NaN  1067514.0    KS
10216  13381.0  Schuyler County Bancshares Inc    NaN  1100055.0    MO
10257  17636.0     First Belleville Bancshares    NaN  1060917.0    KS
10445   1756.0     Cross County Bancshares Inc    NaN  1131611.0    AR
10539  57197.0                     Bnccorp Inc   BNCC  1248162.0    ND

In [62]:
bhcc[bhcc['xCert']==17399]

xCert          Parent Ticker    ParRSSD ParST
6777  17399.0  First Fin Corp    NaN  1067514.0    KS
9819  17399.0  First Fin Corp    NaN  1067514.0    KS

In [63]:
len(bhcc) # 4190

4190

In [64]:
bhcc2 = bhcc.drop_duplicates(subset=['xCert','Parent','Ticker','ParRSSD'],keep=False)
len(bhcc2) # 4150

4150

In [65]:
bhcc2.head()

xCert                Parent Ticker    ParRSSD ParST
0   628.0     Jpmorgan Chase&Co    JPM  1039502.0    NY
1  3510.0  Bank Of America Corp    BAC  1073757.0    NC
2  7213.0         Citigroup Inc      C  1951350.0    NY
3  3511.0   Wells Fargo&Company    WFC  1120754.0    CA
5  6548.0             U S Bcorp    USB  1119794.0    MN

In [66]:
bhcc2.set_index(['xCert'],drop=True,inplace=True)
bhcc2.head()

Parent Ticker    ParRSSD ParST
xCert                                               
628.0      Jpmorgan Chase&Co    JPM  1039502.0    NY
3510.0  Bank Of America Corp    BAC  1073757.0    NC
7213.0         Citigroup Inc      C  1951350.0    NY
3511.0   Wells Fargo&Company    WFC  1120754.0    CA
6548.0             U S Bcorp    USB  1119794.0    MN

In [67]:
bhcc2.groupby(level=0).filter(lambda x: len(x) > 1)  ## check for duplicates in xCert index

Empty DataFrame
Columns: [Parent, Ticker, ParRSSD, ParST]
Index: []

In [68]:
dcrt = bhcc2[['Parent','Ticker','ParRSSD','ParST']].to_dict()
dcrt['Parent'][628]

'Jpmorgan Chase&Co'

In [69]:
##### LOOK AT THIS!!! IMPORTANT CONDITIONAL PROCESSING FOR MISSING VALUES!!!!  ## improved 5/9/2016
##########################################################################
j = 0
rk5c = rk5.copy()
for i in np.arange(len(rk5c)):
  if (pd.isnull(rk5c.ix[i]['Parent']))|(rk5c.ix[i]['Parent']==' '):
    xc = rk5c.ix[i]['UltCert']
    try:
      rk5c.loc[i,'Parent'] = dcrt['Parent'][xc]
      rk5c.loc[i,'Ticker'] = dcrt['Ticker'][xc]
      rk5c.loc[i,'ParRSSD'] = dcrt['ParRSSD'][xc]
      rk5c.loc[i,'ParST'] = dcrt['ParST'][xc]
      j = j+1
    except:
      pass

In [70]:
len(rk5c[rk5c['Ticker']>'']) # 3446

3446

In [71]:
j # 4892

4892

In [72]:
len(rk5c) # 10792

10792

In [73]:
rk5c.to_csv("BkAvgAstPR_FailDate_BHC2_ult202003.csv")

In [74]:
# I use ZION to show how many historic banks and mergers got collapsed into one bank charter

In [75]:
rk4.loc[rk4.Ticker.isin(['ZION']),['IDRSSD','xName','xCert','Ticker','Parent','ParRSSD']] # current bank

IDRSSD                     xName   xCert Ticker Parent    ParRSSD
78  276579  ZIONS BANCORPORATION, NA  2270.0   ZION   ZION  1027004.0

In [76]:
rk5c.loc[rk5c.Ticker.isin(['ZION']),['IDRSSD','xName','xCert','Ticker','Parent','ParRSSD']] #historic banks merged in Zions 

IDRSSD                                 xName    xCert Ticker Parent  \
78     276579              ZIONS BANCORPORATION, NA   2270.0   ZION   ZION   
159    837260               CALIFORNIA BANK & TRUST  20852.0   ZION   ZION   
164    676656                        AMEGY BANK, NA  24107.0   ZION   ZION   
316   1004368              NATIONAL BANK OF ARIZONA  20626.0   ZION   ZION   
342    456960                     NEVADA STATE BANK  18113.0   ZION   ZION   
455    933957              VECTRA BANK COLORADO, NA   2993.0   ZION   ZION   
731    807665                     VINEYARD BANK, NA  23556.0   ZION   ZION   
1134   458263                  STOCKMEN'S BANK, THE  23182.0   ZION   ZION   
1184  1198641  COMMERCE BANK OF WASHINGTON, NA, THE  27298.0   ZION   ZION   
1250  2479310                     SILVER STATE BANK  34194.0   ZION   ZION   
1687   720465                         ALLIANCE BANK  23124.0   ZION   ZION   
1762   618656                            KLEIN BANK  22866.0   ZION   ZION   
2795   105950          MINNEQUA BANK OF PUEBLO, THE   1773.0   ZION   ZION   
3270   576158                             MAXIMBANK   9017.0   ZION   ZION   
4196   309664                           RANCHO BANK  22934.0   ZION   ZION   
4287   521550                        LONE STAR BANK  22304.0   ZION   ZION   
4763  2068134            GREAT BASIN BANK OF NEVADA  33824.0   ZION   ZION   
6292   626754              LEAGUE CITY BANK & TRUST  17577.0   ZION   ZION   
6484  3161993                           CHOICE BANK  57469.0   ZION   ZION   
6772   687269        INTERCONTINENTAL NATIONAL BANK  19112.0   ZION   ZION   
6807   456670                   FRONTIER STATE BANK  24227.0   ZION   ZION   
8471  3374186          COMMERCE BANK OF OREGON, THE  58223.0   ZION   ZION   
9935  2907448               SOUTHLAND BUSINESS BANK  35287.0   ZION   ZION   

        ParRSSD  
78    1027004.0  
159   1027004.0  
164   1027004.0  
316   1027004.0  
342   1027004.0  
455   1027004.0  
731   1027004.0  
1134  1027004.0  
1184  1027004.0  
1250  1027004.0  
1687  1027004.0  
1762  1027004.0  
2795  1027004.0  
3270  1027004.0  
4196  1027004.0  
4287  1027004.0  
4763  1027004.0  
6292  1027004.0  
6484  1027004.0  
6772  1027004.0  
6807  1027004.0  
8471  1027004.0  
9935  1027004.0